In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import xgboost as xgb
pd.set_option('display.max_colwidth', -1)

In [2]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [9]:
from utils import UtilsKy, Statistic
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [15]:
# kyw3
# path_data = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/04_experiments/ex_01_some_teach/'
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

# db_teach = pd.read_csv(path_data + 'db_teach_kyw3_with_factors.csv', dtype=str)
# db_test = pd.read_csv(path_data + 'db_test_kyw3_with_factors.csv', dtype=str)

In [22]:
# ky9
path_data = '/mnt/files/workdata/work/merchants/merchant_32_ky9_2020-05-12_white_visa/04_experiments/'
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KY9, dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

# for prod
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9_FOR_PROD, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KY9_FOR_PROD , dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

db_teach = pd.read_csv(path_data + 'ex_05_ky9_xgb_jupiter_2020_07_08/db_teach_ky9_w_len_phone_st.csv', dtype=str)
db_test = pd.read_csv(path_data + 'ex_05_ky9_xgb_jupiter_2020_07_08/db_test_ky9_w_len_phone_st.csv', dtype=str)

In [23]:
db_teach.columns

Index(['amount', 'bank_currency', 'bin', 'client_hour', 'date_cb_only',
       'date_only', 'day_of_week', 'hour', 'id',
       'is_bank_country_equal_country', 'is_ip_country_equal_country',
       'latitude', 'longitude', 'order_id', 'phone_2_norm', 'status',
       'status_chargeback', 'time', 'is_city_resolved', 'country_code3',
       'country', 'ip', 'network', 'num_network', 'is_usa', 'is_can', 'is_bra',
       'distance_l2', 'distance_hav', 'distance_sub', 'gender',
       'amount_deviation', 'count_months_to_end_card', 'dgender', 'is_feman',
       'is_man', 'is_undef', 'is_phone_standart', 'amount_cb',
       'is_status_bad'],
      dtype='object')

In [24]:
db_teach.shape[0]

742167

In [25]:
db_teach[db_teach.status.isin([1, '1', 'true'])].shape[0]

42005

In [81]:
COL_FACTORS = [ 'amount', 'bank_currency', 'hour', 'day_of_week', 
               'bin', 'longitude', 'latitude', 'phone_2_norm','is_phone_standart', 'is_city_resolved', 'is_undef']

In [82]:
train = db_teach[COL_FACTORS]
test = db_test[COL_FACTORS]

In [83]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [84]:
replace_val = -9999
train.fillna(replace_val)
test.fillna(replace_val)
replace_val

-9999

In [85]:
train = train.values
test = test.values
label = db_teach.status

In [86]:
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [87]:
#result_df_amount = None
weight = analyzer_prediction.get_xgb_weight()

In [88]:
# Kyw3
# max_depths =[3]
# nrounds = [80, 90]
# etas = [0.2, 0.35]

# Ky9
max_depths =[5]
nrounds = [70, 90]
etas = [0.35, 0.4]

In [89]:
for nround in nrounds:
    for eta in etas:
        for max_depth in max_depths:
            config = {'max_depth': max_depth, 'learning_rate': eta
                      , 'n_estimators': nround
                     }
            model = xgb.XGBClassifier(**config)
            
            model.fit(train, label, eval_metric = 'auc', sample_weight=weight)
             
            test_pred = model.predict_proba(test)
            db_test["probability"] = test_pred[:, 1]

            description = '-' . join([str(elem) for elem in (max_depth, nround, eta, 'is_phone_standart', 'is_undef','is_city_resolved')])                    
            result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")

In [90]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ] 
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)
stat_best = stat_best.sort_values(by="rating", ascending=False)

In [91]:
stat_best.iloc[:,:11]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
2,5-70-0.4,7.45,13.73,18.97,24.71,29.06,33.88,38.08,51.02,79.90,224.84
6,5-90-0.4,6.97,13.29,18.71,24.62,29.74,34.76,38.78,51.32,79.68,224.33
0,5-70-0.35,8.99,14.20,19.52,23.30,27.94,32.23,37.34,49.33,79.17,222.45
4,5-90-0.35,9.24,14.70,19.12,22.78,26.88,31.69,36.65,48.92,79.71,221.35
12,5-90-0.35,5.14,8.81,11.51,14.33,15.99,18.10,19.54,25.46,46.31,127.55
28,5-90-0.35-is_phone_standart,4.90,7.88,10.49,13.13,15.40,17.66,20.23,27.86,45.88,125.54
8,5-70-0.35,4.59,8.32,10.65,13.81,15.57,18.08,19.46,25.72,46.73,125.39
42,5-70-0.4-is_phone_standart-is_city_resolved,4.27,7.20,10.12,12.84,15.29,17.97,20.38,26.75,48.41,124.88
46,5-90-0.4-is_phone_standart-is_city_resolved,3.54,7.32,9.87,13.01,15.28,18.27,20.99,28.06,47.21,124.29
24,5-70-0.35-is_phone_standart,4.08,7.38,9.99,12.51,14.98,16.99,20.19,26.45,45.83,121.22
